In [10]:
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import os
from matplotlib.patches import Patch

def get_direction_code(img_sitk):
    """ Get the direction of a SimpleITK image.
    Args:
        img_sitk (SimpleITK.Image): Input image.
    Returns:
        str: Direction code of the image.
    """
    direction_code = sitk.DICOMOrientImageFilter().GetOrientationFromDirectionCosines(img_sitk.GetDirection())

    return direction_code

def reorient_sitk(img_sitk, new_direction):
    """ Reorient a SimpleITK image to a new direction.
    Args:
        img_sitk (SimpleITK.Image): Input image to be reoriented.
        new_direction (str): New direction code (e.g., 'LPS', 'RAS').
    Returns:
        SimpleITK.Image: Reoriented image.
    """

    img_sitk_reoriented = sitk.DICOMOrient(img_sitk, new_direction)


In [12]:
import os
from tqdm import tqdm
import numpy as np
import SimpleITK as sitk

def dice_score(y_true, y_pred, smooth=1e-6):
    """
    Compute the Dice score between two binary masks.

    Args:
        y_true (numpy.ndarray): Ground truth binary mask.
        y_pred (numpy.ndarray): Predicted binary mask.
        smooth (float): Smoothing factor to avoid division by zero.

    Returns:
        float: Dice score.
    """
    intersection = np.sum(y_true * y_pred)
    return (2. * intersection + smooth) / (np.sum(y_true) + np.sum(y_pred) + smooth)

label_gt_path = "/scratch/awias/data/Pancreas/nnUNet_dataset/nnUNet_raw/Dataset001_Pancreas/labelsTs"
label_pred_path = "/scratch/awias/data/Pancreas/nnUNet_dataset/nnUNet_raw/Dataset001_Pancreas/imagesTs/man_preds"

label_gt_file = [x for x in os.listdir(label_gt_path) if x.endswith(".nii.gz")]
label_pred_file = [x for x in os.listdir(label_pred_path) if x.endswith(".nii.gz")]

all_subjects = [x.split('_')[0] for x in label_pred_file]

#Make sure the files are sorted in the same order
label_gt_file = sorted(label_gt_file, key=lambda x: x.split('_')[0])
label_pred_file = sorted(label_pred_file, key=lambda x: x.split('_')[1])

# if len(label_gt_file) != len(label_pred_file):
#     raise ValueError("Number of ground truth and prediction files do not match")

output_folder = "/scratch/awias/data/Pancreas/test"

dice_list = []
n = len(label_gt_file)

for subject in all_subjects:
    try:
        mask_gt_sitk = sitk.ReadImage(os.path.join(label_gt_path, f"{subject}.nii.gz"))
        pred_sitk = sitk.ReadImage(os.path.join(label_pred_path, f"{subject}_0000_pred.nii.gz"))
        
        print(get_direction_code(mask_gt_sitk))
        print(get_direction_code(pred_sitk))
        
        pred_sitk.CopyInformation(mask_gt_sitk)  # Ensure the prediction has the same metadata as the ground truth
        
        # Save mask_gt_sitk and pred_sitk with new orientation in same folder
        sitk.WriteImage(mask_gt_sitk, os.path.join(output_folder, f"{subject}_reoriented.nii.gz"))
        sitk.WriteImage(pred_sitk, os.path.join(output_folder, f"{subject}_0000_pred_reoriented.nii.gz"))

        print("Saved")

        mask_gt = sitk.GetArrayFromImage(mask_gt_sitk)
        pred = sitk.GetArrayFromImage(pred_sitk)
        
        print(get_direction_code(mask_gt_sitk))
        print(get_direction_code(pred_sitk))

        # print(os.path.join(label_gt_path,label_gt_file[i]))
        # print(os.path.join(label_pred_path,label_pred_file[i]))
        # plot_central_slice_img_mask(mask_gt, pred, spacing=None)
        

        dice = dice_score(mask_gt, pred)
        dice_list.append(dice)
        
        print(f"Dice score for subject {subject} is: {dice}")
    except:
        print(f"Error processing subject {subject}. Skipping...")

print(f"Mean dice score: {sum(dice_list) / len(dice_list)}")
print(f"Median dice score: {sorted(dice_list)[len(dice_list) // 2]}")
print(f"Max dice score: {max(dice_list)}")
print(f"Min dice score: {min(dice_list)}")


LAI
LAS
Error processing subject 38. Skipping...
LAI
LAS
Error processing subject 13. Skipping...
LAI
LAS
Error processing subject 39. Skipping...
LAI
LAS
Error processing subject 72. Skipping...


** ERROR (nifti_image_write_engine): cannot open output file '/scratch/awias/data/Pancreas/test/38_reoriented.nii.gz'
** ERROR (nifti_image_write_engine): cannot open output file '/scratch/awias/data/Pancreas/test/13_reoriented.nii.gz'
** ERROR (nifti_image_write_engine): cannot open output file '/scratch/awias/data/Pancreas/test/39_reoriented.nii.gz'
** ERROR (nifti_image_write_engine): cannot open output file '/scratch/awias/data/Pancreas/test/72_reoriented.nii.gz'


LAI
LAS
Error processing subject 73. Skipping...
LAI
LAS
Error processing subject 1. Skipping...
LAI
LAS
Error processing subject 75. Skipping...
LAI
LAS
Error processing subject 51. Skipping...


** ERROR (nifti_image_write_engine): cannot open output file '/scratch/awias/data/Pancreas/test/73_reoriented.nii.gz'
** ERROR (nifti_image_write_engine): cannot open output file '/scratch/awias/data/Pancreas/test/1_reoriented.nii.gz'
** ERROR (nifti_image_write_engine): cannot open output file '/scratch/awias/data/Pancreas/test/75_reoriented.nii.gz'
** ERROR (nifti_image_write_engine): cannot open output file '/scratch/awias/data/Pancreas/test/51_reoriented.nii.gz'


LAI
LAS
Error processing subject 19. Skipping...
LAI
LAS
Error processing subject 27. Skipping...
LAI
LAS
Error processing subject 30. Skipping...
LAI
LAS
Error processing subject 55. Skipping...


** ERROR (nifti_image_write_engine): cannot open output file '/scratch/awias/data/Pancreas/test/19_reoriented.nii.gz'
NiftiImageIO (0x4191ee0): /scratch/awias/data/Pancreas/nnUNet_dataset/nnUNet_raw/Dataset001_Pancreas/labelsTs/27.nii.gz has unexpected scales in sform

NiftiImageIO (0x4191ee0): /scratch/awias/data/Pancreas/nnUNet_dataset/nnUNet_raw/Dataset001_Pancreas/labelsTs/27.nii.gz has unexpected scales in sform

** ERROR (nifti_image_write_engine): cannot open output file '/scratch/awias/data/Pancreas/test/27_reoriented.nii.gz'
** ERROR (nifti_image_write_engine): cannot open output file '/scratch/awias/data/Pancreas/test/30_reoriented.nii.gz'
** ERROR (nifti_image_write_engine): cannot open output file '/scratch/awias/data/Pancreas/test/55_reoriented.nii.gz'


LAI
LAS
Error processing subject 36. Skipping...
LAI
LAS
Error processing subject 40. Skipping...
LAI
LAS
Error processing subject 20. Skipping...
LAI
LAS
Error processing subject 42. Skipping...


** ERROR (nifti_image_write_engine): cannot open output file '/scratch/awias/data/Pancreas/test/36_reoriented.nii.gz'
** ERROR (nifti_image_write_engine): cannot open output file '/scratch/awias/data/Pancreas/test/40_reoriented.nii.gz'
** ERROR (nifti_image_write_engine): cannot open output file '/scratch/awias/data/Pancreas/test/20_reoriented.nii.gz'
** ERROR (nifti_image_write_engine): cannot open output file '/scratch/awias/data/Pancreas/test/42_reoriented.nii.gz'


ZeroDivisionError: division by zero